# The random phase approximation (RPA)


TO BE WRITTEN : (material below)


- Recall the formula for RPA


- TODO : 
 
 - we grab chi(q,omega)
-------------------------------------------


The Lindhardt susceptibility $\chi_0$ is the exeact susceptibility for the non-interacting case $U=0$, however for finite interactions the susceptibility of the system $\chi$ is given by the Behte-Salpeter equation

$$ \chi = \chi_0 + \chi_0 \Gamma \chi $$

where $\Gamma$ is the particle-hole irreducible vertex function, containing all diagrams with insertions of the interaction that can not be separated by cutting a pair of particle-hole single-particle propagators $G G$.

The first order contribution to the vertext $\Gamma$ is the bare interaction $U$ and the approximation

$$ \Gamma \approx U $$

gives the so-called random phase approximation for $\chi$, i.e.

$$ \chi_{RPA} = \chi_0 + \chi_0 U \chi_{RPA} $$

Rewriting this equation gives $\chi_{RPA}$ as

$$ \chi_{RPA} = \frac{\chi_0}{1 - U \chi_0} $$

we note that the denominator of this equation can in general go to zero, whereby the susceptibility $\chi_{RPA}$ diverges. Whence already the RPA approximation can be used to compute instabilities of the system towards, e.g., anti-ferromagnetic symmetry breaking.

As an example we compute $\chi_{RPA}$ for the square lattice and the enhancement of the $k = (\pi, \pi)$ peak as a function of $U$.

## Load Lindhard chi from archive

In [ ]:
from pytriqs.archive import HDFArchive
from pytriqs.gf import Gf, Idx, inverse

with HDFArchive("tpsc.h5",'r') as R:
    chi0_kw = R['chi0_kw']

## Plot the RPA susceptibility

- Compute the RPA susceptibility for $U = 2.7$
- Plot it

In [ ]:
%matplotlib inline
from pytriqs.plot.mpl_interface import plt
from pytriqs.gf import inverse
import numpy as np

nk = chi0_kw.mesh[0].linear_dims[0]+1
k = np.linspace(0, 2*np.pi, nk, endpoint=True)
kx, ky = np.meshgrid(k, k)

# Note that I can avoid the cell above by just doing
U = 2.7
chi = lambda kx, ky: (chi0_kw([kx,ky,0],0) / (1. - U * chi0_kw([kx,ky,0],0))).real

# 2d plot
plt.figure(figsize=(3.25*2, 5))
plt.title('Static RPA susceptibility $\chi_\mathrm{RPA}(k, \omega=0)$')
plt.pcolor(kx, ky, np.vectorize(chi)(kx,ky))
plt.xlim(0, 2*np.pi)
plt.ylim(0, 2*np.pi)
plt.xticks([0, np.pi, 2*np.pi],[r"0",r"$\pi$",r"$2\pi$"])    
plt.yticks([0, np.pi, 2*np.pi],[r"0",r"$\pi$",r"$2\pi$"])
plt.colorbar()
plt.tight_layout()
plt.xlabel(r'$k_x$')
plt.ylabel(r'$k_y$')


# Here is a 3d plot. With "matplotlib inline" one cannot interact
from mpl_toolkits.mplot3d import axes3d, Axes3D
fig = plt.figure()
ax = Axes3D(fig)
ax.plot_surface(kx, ky, np.vectorize(chi)(kx,ky), cmap="jet", rstride=8, cstride=8)
ax.set_xlabel(r"$k_x$")
ax.set_ylabel(r"$k_y$")
ax.set_zlabel(r"$\chi_\mathrm{RPA}$")


## Plot along some path for different values of $U$ (TODO)

In [ ]:
# CLEAN THIS TO MAKE IT MORE ELEGANT!

kx = np.concatenate([np.linspace(0, np.pi, 100), np.ones(100) * np.pi, np.linspace(np.pi, 0, 100)])
ky = np.concatenate([np.linspace(0, np.pi, 100), np.linspace(np.pi, 0, 100), np.zeros(100)])

plt.figure(figsize=(6,4))
plt.title('Static Lindhardt susceptibility $\chi_0(k,\omega=0)$')

for U in np.arange(1., 2.8, 0.2):
    chiU = lambda kx, ky: (chi0_kw([kx,ky,0],0) / (1. - U * chi0_kw([kx,ky,0],0))).real
    plt.plot(np.arange(300), np.vectorize(chiU)(kx,ky), label=r'$\chi_\mathrm{RPA}$, $U=%2.2f$' % U)

plt.grid();
plt.axes().set_xticks([0,100,200,300])
plt.axes().set_xticklabels([r'$\Gamma$',r'$X$',r'$M$',r'$\Gamma$']);
plt.ylabel(r'$\chi_0(k,\omega=0)$')
plt.legend(loc='best')
plt.tight_layout()

## Find critical $U$

At some critical $U_c$ the susceptibility diverges $\chi \rightarrow \infty$ within the random phase approximation. To determine $U_c$ we can study the root of the inverse susceptibility $\chi_{RPA}^{-1}$.

For the square lattice it is sufficient to study the response at $k_{AF}= (\pi, \pi)$ since it is the point where the response diverges. Analytically this occurs when the denominator is zero $1 - U_c \chi_0(0, k_{AF}) = 0$, i.e.

$$ U_c^{(RPA)} = \frac{1}{\chi_0(0, k_{AF})} $$

numerically this looks like

In [ ]:
# Critical U - analytical solution
U_c = 1 / chi0_kw([np.pi,np.pi,0], 0).real

Ur = np.linspace(0,4,100)
chi_inv = lambda U: ((1. - U * chi0_kw([np.pi, np.pi, 0],0)) / chi0_kw([np.pi, np.pi, 0], 0)).real

plt.figure(figsize=(6, 4))
plt.title("Determination of critical $U$ in RPA")
plt.plot(Ur, chi_inv(Ur), label=r'$\chi_{RPA}^{-1}$')
plt.plot(Ur, np.zeros_like(Ur), 'k', lw=0.5)
plt.plot(U_c, 0, 'rs', label=r'$U_c \approx %2.2f$'%U_c, alpha=0.5)
plt.legend(loc='best')
plt.xlabel(r'$U$')
plt.ylabel(r'$\chi_\mathrm{RPA}^{-1}$')

## Illustration of speed difference (to be removed)

In [ ]:
def chi_rpa_1(chi0, U):
    chi_rpa = chi0.copy()
    kmesh, wmesh = chi0.mesh.components
    for k in kmesh:
        for w in wmesh:
            chi_rpa[k,w] = chi0_kw[k,w] / (1. - U * chi0_kw[k,w])
    return chi_rpa

def chi_rpa_2(chi0, U):
    chi_rpa = chi0.copy()
    kmesh, wmesh = chi0.mesh.components
    for k in kmesh:
        chi_rpa[k,:] << inverse(1. - U * chi0_kw[k,:]) * chi0_kw[k,:]
    return chi_rpa

def chi_rpa_3(chi0, U):
    chi_rpa = chi0.copy()
    chi_rpa = chi0 * inverse(1 - U * chi0)
    return chi_rpa

In [ ]:
%%timeit
U = 2.7
chi_rpa_1(chi0_kw, U)

In [ ]:
%%timeit
U = 2.7
chi_rpa_2(chi0_kw, U)

In [ ]:
%%timeit
U = 2.7
chi_rpa_3(chi0_kw, U)